In [ ]:
import os
import json
import copy
import time
import tqdm
import torch
import numpy as np
import torchvision
import pandas as pd
import torch.nn as nn
from skimage import io
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from random import shuffle
from torchvision import transforms,datasets, models
from torch.utils.data import Dataset
from IPython.display import clear_output
print(torch.__version__) # This code has been updated for PyTorch 1.0.0

In [ ]:
torch.cuda.empty_cache()

In [ ]:
labelsTable = pd.read_csv('/content/drive/MyDrive/modefied_labels.csv')
labelsTable=labelsTable.to_json()
labelsTable=json.loads(labelsTable)
allLabels={}
for row in range(35108):
    allLabels[labelsTable['image'][str(row)]]=labelsTable['level'][str(row)]
print(allLabels)

In [ ]:
# Preparing train list
trainShuffList = os.listdir('/content/drive/MyDrive/trainData')
trainLabelShuffList = []
for img in trainShuffList:
  trainLabelShuffList.append(allLabels[img[:-5]])
print(len(trainShuffList))
print(len(trainLabelShuffList))

In [ ]:
# Preparing test list
testShuffList = os.listdir('/content/drive/MyDrive/testData')
testLabelShuffList = []
for img in testShuffList:
  testLabelShuffList.append(allLabels[img[:-5]])
print(testShuffList)
print(len(testLabelShuffList))

## Load Data:

In [ ]:
class customDataLoader(Dataset):
  def __init__(self,root_dir,labelList,itemList,transforms=None):
    self.itemList=itemList
    self.labelList=labelList
    self.root_dir=root_dir
    self.transforms=transforms 

  def __len__(self):
    return len(self.itemList)
  
  def __getitem__(self,index):
      img_path=os.path.join(self.root_dir,self.itemList[index-1])
      image = Image.open(img_path)
      y_label=self.labelList[index-1]
      if(self.transforms):
        image = self.transforms(image)
    
      return (image,y_label)



In [ ]:
trainDir='/content/drive/MyDrive/trainData'
testDir='/content/drive/MyDrive/testData'
apply_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
train_dataset = customDataLoader(trainDir,trainLabelShuffList,trainShuffList,apply_transform)
trainLoader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
test_dataset = customDataLoader(testDir,testLabelShuffList,testShuffList,apply_transform)
testLoader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
# Size of train and test datasets
print('No. of samples in train set: '+str(len(trainLoader.dataset)))
print('No. of samples in test set: '+str(len(testLoader.dataset)))

## Define network architecture
Define your architecture and modify classification/fullyConected output accordingly

In [ ]:
net = models.resnet18()
# Modifying the last fully-connected layer for 10 classes
net.fc = nn.Linear(512,5)
#print(net)

 Change runtime of google colab to gpu

In [ ]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()
# use_gpu = False # Uncomment in case of GPU memory error
if use_gpu:
    print('GPU is available!')
    device = "cuda"
else:
    print('GPU is not available!')
    device = "cpu"
    
net = net.to(device)

GPU is available!


Declare variables to store epoch accuracy and loss 

In [ ]:
trainDataTensor=[]
testDataTensor=[]

In [ ]:
trainLoss = []
trainAuxLoss = []
trainTotalLoss = []
trainAcc = []
testLoss = []
testAcc = []

## Define loss function and optimizer and  Train the network.

In [ ]:
# Testing Input to Network
BatchSize=64
criterion = nn.NLLLoss() # Negative Log-likelihood
optimizer = optim.Adam(net.parameters(), lr=1e-5) # Adam

for epoch in range(1000):
  runningLoss = 0.0   
  avgTotalLoss = 0.0
  running_correct = 0
  net.train()
  for data in trainLoader:
    try:
      trainDataTensor.append(data)
      input,labels=data
      input=input.to(device)
      labels=labels.to(device)
      optimizer.zero_grad()
      outputs = net(input) 
      print(outputs,labels)
      _, predicted = torch.max(outputs.data, 1)  
      print(predicted,labels)
      running_correct += (predicted == labels.data).sum()           
            # Compute loss/error
      #print(labels.size())
      loss = criterion(F.log_softmax(outputs,dim=1), labels.to(torch.long))
            # Backpropagate loss and compute gradients
      loss.backward()
            # Update the network parameters
      optimizer.step()
            # Accumulate loss per batch
      runningLoss += loss.item()
    except Exception as e:
      print(e.message)
  print(running_correct)
  avgTrainAcc = 100*float(running_correct)/10048
  avgTrainLoss = runningLoss/(10048/BatchSize)    
  trainAcc.append(avgTrainAcc)
  trainLoss.append(avgTrainLoss)
  print(epoch,'/',10,'avgTrainAcc:',avgTrainAcc,'avgTrainLoss',avgTrainLoss)

  net.eval() # For testing [Affects batch-norm and dropout layers (if any)]
  running_correct = 0
  with torch.no_grad():
      for data in testLoader:
          testDataTensor.append(data)
          inputs,labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = net(inputs)
          
          _, predicted = torch.max(outputs.data, 1)
          running_correct += (predicted == labels.data).sum()   
          loss = criterion(F.log_softmax(outputs,dim=1), labels.to(torch.long))
          runningLoss += loss.item() 
  
            
  avgTestLoss = runningLoss/(2560/BatchSize)
  avgTestAcc = 100*float(running_correct)/2560
  print(epoch,'/',10,'avgTestAcc:',avgTestAcc,'avgTrainLoss',avgTestLoss)
  testLoss.append(avgTestLoss)
  testAcc.append(avgTestAcc)
  if(1):
    break

In [ ]:
# Testing Input to Network
BatchSize=64
criterion = nn.NLLLoss() # Negative Log-likelihood
optimizer = optim.Adam(net.parameters(), lr=1e-1) # Adam
for epoch in range(1000):
  runningLoss = 0.0   
  avgTotalLoss = 0.0
  running_correct = 0
  net.train()
  for data in trainDataTensor :
    try:
      input,labels=data
      input=input.to(device)
      labels=labels.to(device)
      optimizer.zero_grad()
      outputs = net(input) 
      _, predicted = torch.max(outputs.data, 1)  
      running_correct += (predicted == labels.data).sum()           
            # Compute loss/error
      #print(labels.size())
      loss = criterion(F.log_softmax(outputs,dim=1), labels.to(torch.long))
            # Backpropagate loss and compute gradients
      loss.backward()
            # Update the network parameters
      optimizer.step()
            # Accumulate loss per batch
      runningLoss += loss.item()
      idk+=1
    except Exception as e:
      print(e.message)
  print(running_correct)
  avgTrainAcc = 100*float(running_correct)/10048
  avgTrainLoss = runningLoss/(10048/BatchSize)    
  trainAcc.append(avgTrainAcc)
  trainLoss.append(avgTrainLoss)
  print(epoch,'/',10,'avgTrainAcc:',avgTrainAcc,'avgTrainLoss',avgTrainLoss)

  net.eval() # For testing [Affects batch-norm and dropout layers (if any)]
  running_correct = 0
  with torch.no_grad():
      for data in testDataTensor:
          inputs,labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = net(inputs)
          _, predicted = torch.max(outputs.data, 1)
          running_correct += (predicted == labels.data).sum()   
          loss = criterion(F.log_softmax(outputs,dim=1), labels.to(torch.long))
          runningLoss += loss.item() 
  
  avgTestLoss = runningLoss/(2560/BatchSize)
  avgTestAcc = 100*float(running_correct)/2560
  print(epoch,'/',10,'avgTestAcc:',avgTestAcc,'avgTrainLoss',avgTestLoss)
  testLoss.append(avgTestLoss)
  testAcc.append(avgTestAcc)

## Visualization of weights

In [ ]:
fig1 = plt.figure(1)        
plt.plot(range(1000),[i  for i in trainLoss],'r',label='trainloss') 
plt.plot(range(1000),[i  for i in testLoss],'g',label='testloss')
plt.xlabel("epoch")
plt.ylabel("loss")
plt.title("loss vs epoch")
plt.legend()


fig2=plt.figure(2)
plt.plot(range(1000),[i  for i in trainAcc],'r',label='trainAcc') 
plt.plot(range(1000),[i  for i in testAcc],'g',label='testAcc') 
plt.xlabel("epoch")
plt.ylabel("Acc")
plt.title("Accuracy vs epoch")
plt.legend()


In [ ]:
torch.save(net.state_dict(), '/content/drive/MyDrive/dr_trained_model.pt')